In [ ]:
# CSV TO PARQUET


import pandas as pd
import os

# Caminhos de entrada e saída
caminho_csv = 'C:\\AFL_GABRIEL\\csv_parser\\NF-e\\sis'
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\NF-e Parquet\\sis'

# Nome do arquivo final Parquet
nome_arquivo_parquet = 'M400'
caminho_arquivo_parquet = os.path.join(caminho_parquet, nome_arquivo_parquet)

# Criar o diretório de saída, se necessário
os.makedirs(caminho_parquet, exist_ok=True)

# Lista para armazenar os DataFrames
lista_dfs = []

# Iterar sobre os arquivos CSV
for arquivo in os.listdir(caminho_csv):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(caminho_csv, arquivo)

        try:
            df = pd.read_csv(caminho_arquivo, delimiter="|", dtype=str)
            if not df.empty:
                lista_dfs.append(df)
                print(f'Lido: {arquivo} com {df.shape[0]} linhas')
            else:
                print(f'Ignorado (vazio): {arquivo}')
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')

# Verifica se há dados para salvar
if lista_dfs:
    df_final = pd.concat(lista_dfs, ignore_index=True)
    try:
        df_final.to_parquet(caminho_arquivo_parquet, index=False)
        print(f'Dados salvos em: {caminho_arquivo_parquet} ({df_final.shape[0]} linhas)')
    except Exception as e:
        print(f'Erro ao salvar o arquivo Parquet: {e}')
else:
    print('Nenhum dado válido encontrado. Nenhum arquivo Parquet foi criado.')

print('Conversão concluída!')


Lido: EFD - Contribuições - 1100 - Controle de Créditos Fiscais - PIS-Pasep.csv com 14 linhas
Dados salvos em: C:\AFL_GABRIEL\csv_parser\NF-e Parquet\sis\1100 (14 linhas)
Conversão concluída!


In [ ]:
# XLSX TO PARQUET

import pandas as pd
import os

# Diretório onde os XLSX estão localizados
pasta_xlsx = 'C:\\AFL_GABRIEL\\csv_parser\\NF-e\\gabarito'  
pasta_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\NF-e Parquet\\gabarito'  

# Criar o diretório de saída, se não existir
if not os.path.exists(pasta_parquet):
    os.makedirs(pasta_parquet)

# Lista para armazenar DataFrames das páginas
dfs_paginas = {}

# Iterar pelos arquivos XLSX no diretório
for arquivo in os.listdir(pasta_xlsx):
    if arquivo.endswith('.xlsx'):
        caminho_arquivo = os.path.join(pasta_xlsx, arquivo)

        try:
            # Lê todas as planilhas do arquivo XLSX em um dicionário
            sheets_dict = pd.read_excel(caminho_arquivo, sheet_name=None)
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')
            continue

        # Extrai o nome base antes da parte de paginação, se houver
        nome_base = arquivo.split('_pagina')[0]

        # Se ainda não tiver uma entrada para este nome_base, cria
        if nome_base not in dfs_paginas:
            dfs_paginas[nome_base] = []

        # Adiciona todos os DataFrames das planilhas à lista
        for nome_planilha, df in sheets_dict.items():
            dfs_paginas[nome_base].append(df)

# Processar e unir os DataFrames por nome_base
for nome_base, lista_dfs in dfs_paginas.items():
    if len(lista_dfs) > 1:
        df_final = pd.concat(lista_dfs, ignore_index=True)
        print(f'{nome_base} - Páginas unidas com {df_final.shape[0]} linhas')
    elif lista_dfs:
        df_final = lista_dfs[0]
    else:
        print(f'{nome_base} - Nenhum dado encontrado')
        continue

    # Caminho para salvar o arquivo Parquet
    caminho_arquivo_parquet = os.path.join(pasta_parquet, f'{nome_base}.parquet')

    try:
        df_final.to_parquet(caminho_arquivo_parquet, index=False)
        print(f'Salvo: {caminho_arquivo_parquet}')
    except Exception as e:
        print(f'Erro ao salvar {nome_base}.parquet: {e}')

print('Conversão concluída!')


Salvo: C:\AFL_GABRIEL\csv_parser\NF-e Parquet\gabarito\MA NF-e.xlsx.parquet com 8097 linhas
✅ Conversão concluída com sucesso!


In [5]:
import pandas as pd
import pyperclip

# Caminho do arquivo Parquet
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\NF-e Parquet\\sis'

# Lista de palavras-chave para conversão e substituição
palavras_chave = ["Vlr", "Valor", "vlr", "valor", "Vr", "vr", 
                  "base", "Base", "Salário", "Salario", "salário", 
                  "salario", "Provento", "provento", "Desconto", "desconto", 
                  "diferenca", "Diferenca", "diferença", "Diferença"]   

# Ler o arquivo Parquet
df = pd.read_parquet(caminho_parquet)

# Informações do DataFrame
num_linhas, num_colunas = df.shape

# Criar a string com os resultados
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

# Percorrer todas as colunas
for coluna in df.columns:
    linhas_preenchidas = df[coluna].count()  # Conta valores não nulos

    # Verifica se a coluna contém palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in palavras_chave):
        # Converte valores da coluna para float (com substituições)
        df[coluna] = (
            df[coluna].astype(str)
            .str.replace('.', '', regex=False)      # Remove separador de milhar
            .str.replace(',', '.', regex=False)     # Converte separador decimal
            .str.extract(r'(\d+\.?\d*)')[0]          # Extrai apenas números
            .astype(float)
        )

        # Filtra valores não nulos e diferentes de zero
        valores_validos = df[coluna][(df[coluna].notna()) & (df[coluna] != 0)]

        # Soma os valores válidos
        soma_valores = valores_validos.sum()

        # Formatação brasileira
        valor_formatado = f"{soma_valores:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

        resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para float)\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
        resultado += f"   🔹 Total Somado: {valor_formatado}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print("\n✅ Resultado copiado para a área de transferência!")



✅ Resultado copiado para a área de transferência!
